<a href="https://colab.research.google.com/github/yuto-kobayashi-1/signate-AIQuest/blob/develop/210716_AIQuest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import lightgbm as lgb
from copy import deepcopy
import numpy as np
from datetime import datetime , date

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [57]:
data = pd.read_csv("/content/drive/MyDrive/signate/competiton/210716_AI Quest/train.csv",encoding="utf_8")
test = pd.read_csv("/content/drive/MyDrive/signate/competiton/210716_AI Quest/test.csv",encoding="utf_8")

In [58]:
def preprocessing(df):
  df["host_response_rate"] = df["host_response_rate"].str.replace('%', '').astype(float)
  df["host_since"] = (date(2021,7,21) - pd.to_datetime(df["host_since"]).dt.date).dt.days
  # df["first_review"] = (date(2021,7,21) - pd.to_datetime(df["first_review"]).dt.date).dt.days
  # df["last_review"] = (date(2021,7,21) - pd.to_datetime(df["last_review"]).dt.date).dt.days
  # df["review_days"] = (data["first_review"] - data["last_review"])
  df['amenities'] = df['amenities'].str.replace('"', '')
  df['amenities'] = df['amenities'].str.replace('{', '')
  df['amenities'] = df['amenities'].str.replace('}', '')
  
  return df

def label_encorder(df_train,df_test,cols):
  df_train["flg"] = 0
  df_test["y"] = 100
  df_test["flg"] = 1

  df_ = pd.concat([df_train,df_test]).reset_index(drop=True)
  le = LabelEncoder()

  for col in cols:
    df_[col] = df_[col].fillna("OK")
    null_list = list(df_[df_[col] == "OK"].index)
    print(len(null_list))
    encoded = le.fit_transform(df_[col].values)
    decoded = le.inverse_transform(encoded)
    df_[col] = encoded
    df_.loc[null_list,col]=np.nan

  df_train_ = df_[df_["flg"]==0].drop("flg",axis=1)
  df_test_ = df_[df_["flg"]==1].drop(["flg","y"],axis=1)

  return df_train_ ,  df_test_

def get_dummys(df_train,df_test,cols):
  df_train["flg"] = 0
  df_test["y"] = 100
  df_test["flg"] = 1

  df_ = pd.concat([df_train,df_test]).reset_index(drop=True)

  amenities = df_["amenities"].str.get_dummies(sep=',')

  df_ = pd.concat([df_,amenities],axis=1)

  df_train_ = df_[df_["flg"]==0].drop("flg",axis=1)
  df_test_ = df_[df_["flg"]==1].drop(["flg","y"],axis=1)

  return df_train_ ,  df_test_


def train(X,y,label_cols):

  kf = KFold(n_splits=5,shuffle=True,random_state=0)

  models = []
  for train_index, test_index in kf.split(X, y):
    X_train = X.iloc[train_index]
    y_train =  y.iloc[train_index]
    X_valid = X.iloc[test_index]
    y_valid = y.iloc[test_index]

    X_train, y_train, X_valid, y_valid =  target_encording(X_train, y_train, X_valid, y_valid,label_cols)

    params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 64,
    'min_data_in_leaf': 20,
    'max_depth': 7,
    'verbose': 0,
  }
    train_data = lgb.Dataset(
      data=X_train, 
      label=y_train,
  )

    validation_data = lgb.Dataset(
      data=X_valid, 
      label=y_valid, 
  )

    model = lgb.train(
      params=params, 
      train_set=train_data, 
      num_boost_round=1000, 
      early_stopping_rounds=100,
      valid_sets=[validation_data]
  )

    models.append(model)

  return models 

def feature_importance(models,data,cols):
  importance = pd.DataFrame(columns=data[cols].columns)

  for model in models:
    imp_ = pd.Series(model.feature_importance(),index=data[cols].columns)
    importance = importance.append(imp_,ignore_index=True)

  importance = importance.mean()

  importance = pd.DataFrame(importance,columns={"importance"}).sort_values("importance",ascending=False)

  importance = pd.DataFrame(importance/importance.sum())
  return importance

In [72]:
def target_encording(tr_x, tr_y, va_x, va_y, cols):

  for c in cols:

    data_tmp = pd.DataFrame({c:tr_x[c],'target':tr_y})
    target_mean = data_tmp.groupby(c)['target'].mean()
    #バリデーションデータのカテゴリを置換
    va_x.loc[:,c] = va_x[c].map(target_mean)

    #学習データの変換後の値を格納する配列を準備
    tmp = np.repeat(np.nan,tr_x.shape[0])
    kf_encording = KFold(n_splits=5,shuffle=True,random_state=72)
    for idx_1, idx_2 in kf_encording.split(tr_x):
      #out-of-foldで各カテゴリにおける目的変数の平均を計算
      target_mean = data_tmp.iloc[idx_1].groupby(c)['target'].mean()
      #変換後の値を一時配列に格納
      tmp[idx_2] = tr_x[c].iloc[idx_2].map(target_mean)

    tr_x.loc[:,c] = tmp

  return tr_x, tr_y, va_x, va_y

def target_encording_test(tr_x, tr_y, va_x, cols):
  for c in cols:
    data_tmp = pd.DataFrame({c:tr_x[c],'target':tr_y})
    target_mean = data_tmp.groupby(c)['target'].mean()
    #バリデーションデータのカテゴリを置換
    va_x.loc[:,c] = va_x[c].map(target_mean)

  return va_x

def predict(models,df):
  preds = np.zeros((len(test),len(models)))
  for n , model in enumerate(models):
    # pred = model.predict(df_test[cols],num_iteration=model.best_iteration)
    pred = model.predict(df.drop("id",axis=1),num_iteration=model.best_iteration)
    preds[:,n] = pred

  score = pd.DataFrame()
  score["id"] = df["id"].astype(int)
  score["pred"] = np.mean(preds,axis=1)
  
  return score

In [60]:
len(data)

55583

In [61]:
len(test)

18528

In [81]:
cols = ['accommodates','bathrooms', 'bed_type', 'bedrooms',
       'beds', 'cancellation_policy', 'city', 'cleaning_fee',
       'host_has_profile_pic', 'host_identity_verified',
       'instant_bookable',
       'latitude', 'longitude', 'neighbourhood', 'number_of_reviews',
       'property_type', 'review_scores_rating', 'room_type',"host_response_rate",
        "host_since","amenities"
        ]

drop_cols = ["id","amenities","description","first_review","last_review","name","thumbnail_url","zipcode"]

label_cols = ['bed_type','cancellation_policy',"city","cleaning_fee","host_has_profile_pic","host_identity_verified","instant_bookable","neighbourhood","property_type","room_type"]


In [63]:
data = preprocessing(data)
test = preprocessing(test)

In [64]:
df_train , df_test  = get_dummys(data,test,label_cols)
# df_train , df_test  = data,test

In [65]:
models = train(df_train.drop(drop_cols+["y"],axis=1),df_train['y'],label_cols)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


[1]	valid_0's rmse: 159.263
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 152.116
[3]	valid_0's rmse: 145.812
[4]	valid_0's rmse: 140.458
[5]	valid_0's rmse: 136.031
[6]	valid_0's rmse: 132.361
[7]	valid_0's rmse: 129.274
[8]	valid_0's rmse: 126.58
[9]	valid_0's rmse: 124.366
[10]	valid_0's rmse: 122.381
[11]	valid_0's rmse: 120.645
[12]	valid_0's rmse: 119.135
[13]	valid_0's rmse: 117.938
[14]	valid_0's rmse: 116.933
[15]	valid_0's rmse: 116.088
[16]	valid_0's rmse: 115.291
[17]	valid_0's rmse: 114.66
[18]	valid_0's rmse: 114.226
[19]	valid_0's rmse: 113.758
[20]	valid_0's rmse: 113.335
[21]	valid_0's rmse: 112.902
[22]	valid_0's rmse: 112.563
[23]	valid_0's rmse: 112.261
[24]	valid_0's rmse: 111.972
[25]	valid_0's rmse: 111.579
[26]	valid_0's rmse: 111.384
[27]	valid_0's rmse: 111.141
[28]	valid_0's rmse: 110.943
[29]	valid_0's rmse: 110.704
[30]	valid_0's rmse: 110.481
[31]	valid_0's rmse: 110.356
[32]	valid_0's rmse: 110.271
[33]	valid_0's rmse:

In [66]:
aucs=[]
for model in models:
  aucs.append(model.best_score["valid_0"]["rmse"])
np.mean(aucs)

104.17107190259549

In [67]:
importance = feature_importance(models,df_train,df_train.drop(drop_cols+["y"],axis=1).columns)
importance[importance["importance"]>0].head(50)

,importance
neighbourhood,0.113292
longitude,0.077171
host_since,0.067260
latitude,0.064571
accommodates,0.053712
number_of_reviews,0.050398
bathrooms,0.048918
review_scores_rating,0.047959
host_response_rate,0.042634
property_type,0.037215


In [68]:
df_test = target_encording_test(df_train.drop(drop_cols+["y"],axis=1),df_train['y'],df_test,label_cols)

In [82]:
drop_cols.remove("id")
score = predict(models,df_test.drop(drop_cols,axis=1))

In [83]:
score

,id,pred
55583,0,236.651511
55584,1,133.770772
55585,2,116.106475
55586,3,126.783461
55587,4,135.369066
...,...,...
74106,18523,186.073809
74107,18524,131.316952
74108,18525,207.335241
74109,18526,128.640531


In [84]:
score.dtypes

id        int64
pred    float64
dtype: object

In [85]:
score.to_csv("/content/drive/MyDrive/signate/competiton/210716_AI Quest/submit.csv",header=False,index=False)